In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append('/home/mehdi/github/LSSutils')
from LSSutils.catalogs.datarelease import cols_dr8_ccdts, fixlabels
import matplotlib.backends.backend_pdf

In [2]:
plt.rc('font', family='serif', size=14)

In [3]:
nnbar = lambda s,r,t,m:f'/home/mehdi/data/formehdi/0.4/clustering/nnbar_{s}_{r}_{t}_{m}.npy'

In [4]:
columns = fixlabels(cols_dr8_ccdts, addunit=True)

In [ ]:
pdfname = 'nnbar_desi_eboss_0.4.pdf'
surveys = ['desi', 'eboss']
regions = ['decaln', 'decals', 'bmzls']
templates = ['ccd', 'ts']

models = ['uni', 'lin', 'quad', 'nn', 'nn-ab']

colors = {'uni':'k',
          'lin':'k',
          'quad':'k',
          'nn':'r',
          'nn-ab':'b'}

linestyles = {'uni':'-',
              'lin':'--',
              'quad':'-.',
              'nn':'-',
              'nn-ab':'-'}

labels = {'uni':'No Correction',
          'lin':'Linear',
          'quad':'Quadratic',
          'nn':'NN',
          'nn-ab':'NN+FS'}

def addnnbar(nnbar_list, ax, adderror=False, **kwargs):
    
    for i, nnbar_i in enumerate(nnbar_list):
        
        if adderror:
            ax[i].errorbar(nnbar_i['bin_avg'],
                           nnbar_i['nnbar'],
                           yerr=nnbar_i['nnbar_err'], **kwargs)
        else:
            ax[i].plot(nnbar_i['bin_avg'],
                       nnbar_i['nnbar'], **kwargs)

            
pdf = matplotlib.backends.backend_pdf.PdfPages(pdfname)  

pdf_i = 0

for i, sur in enumerate(surveys):
    print(f'working on {sur}')
    
    for j, reg in enumerate(regions):
        print(f'working on {reg}')
        for k, temp in enumerate(templates):

            # --- set up the plot
            fig, ax = plt.subplots(ncols=3, nrows=10, figsize=(18, 40),
                                  sharey=True)
            fig.subplots_adjust(hspace=0.4, wspace=0.1)
            ax = ax.flatten()

            # plot mean density vs systematics
            for m, mod in enumerate(models):
                nnbar_m = np.load(nnbar(sur, reg, temp, mod), allow_pickle=True)
                
                adderror = True if mod=='uni' else False
                
                addnnbar(nnbar_m, ax, adderror=adderror, color=colors[mod],
                        ls=linestyles[mod], label=labels[mod])

            #--- cosmetics    
            # add x-axes labels
            for m, xlabel in enumerate(columns):
                ax[m].set_ylim(0.9, 1.1)
                ax[m].set_xlabel(xlabel)
                ax[m].grid(True, ls=':', color='grey')
                ax[m].locator_params(tight=True, nbins=5)
                
                if m%3==0:ax[m].set_ylabel(r'$N/\overline{N}$')
                
            # legend    
            title = ' '.join([sur, reg, temp])
            title = title.upper()
            
            ax[0].legend(ncol=3,frameon=False,
                        bbox_to_anchor=(0, 1.1, 3, 0.4), loc="lower left",
                        mode="expand", borderaxespad=0,
                        title=title)
                
            fig.delaxes(ax[-1])    
            pdf.savefig((pdf_i+1), bbox_inches='tight')
            pdf_i += 1
            
pdf.close()      
print('done!')
#plt.show()